<a href="https://colab.research.google.com/github/AnkitaSavaliya/AIH/blob/main/LLM_Diabetes_Prediction_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Standard Libraries
import pandas as pd
from datetime import datetime
import warnings
import numpy as np

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

# Data Resampling
from sklearn.utils import resample

# Suppress warnings to avoid clutter in the output
warnings.filterwarnings('ignore')

# OpenAI API (if required)
import openai

In [11]:
# Mount Google Drive (only needed in Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Predicting Diabetes Using One-Shot, Few-Shot, Chain of Thought, and Decision Tree Approaches**

In [12]:
openClient = openai.OpenAI(api_key="<Replace-with-key>") # As this key is private, removed it while uploading assignment.

In [13]:
# One shot prompt
messages = [
    {"role": "system", "content": "You are a doctor or an expert trained to predict Diabetes based on patient data. Respond with 'Yes' or 'No' with reasoning."},
    {"role": "user", "content": "Predict diabetes risk for a patient with age: 46, gender: Male, BMI: 31, glucose: 88, hypertension: Yes."}
]

response = openClient.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

print(response.choices[0].message.content)

Yes. 

Reasoning: The patient's age (46) and BMI (31) classify him as obese, which significantly increases the risk of developing type 2 diabetes. Additionally, having hypertension is another risk factor for diabetes. While the glucose level of 88 is within the normal range, the combination of obesity and hypertension raises the overall risk of diabetes for this patient.


In [14]:
# few shots
messages = [
    {"role": "system", "content": "You are a doctor or an expert trained to predict Diabetes risk based on patient data. Respond with 'Yes' or 'No' only, and explain your reasoning."},
    {"role": "user", "content": """Here are a few patient cases. Predict if the patient has diabetes and explain your reasoning:

    Case 1: Age: 60, Gender: Male, BMI: 35, Glucose: 180, Hypertension: Yes → Diabetes: Yes

    Case 2: Age: 45, Gender: Female, BMI: 24, Glucose: 90, Hypertension: No → Diabetes: No

    Case 3: Age: 50, Gender: Male, BMI: 31, Glucose: 160, Hypertension: Yes → Diabetes: Yes

    Predict diabetes risk for a new patient: Age: 46, gender: Male, BMI: 31, glucose: 88, hypertension: Yes.. Prediction: Diabetes: """}
]

response = openClient.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

print(response.choices[0].message.content)


No


In [15]:
#Chain of Thought
messages = [
    {"role": "system", "content": "You are a doctor or an expert trained to predict Diabetes risk based on patient data. Respond with 'Yes' or 'No' only, and provide a detailed explanation for the reasoning behind the prediction."},
    {"role": "user", "content": """Here is a patient case. Predict diabetes risk and explain your reasoning:

    Age: 46, Gender: Male, BMI: 31, Glucose: 88, Hypertension: Yes."""},
    {"role": "assistant", "content": """Let's analyze the risk factors step by step:

    - **Age (46)**: While being over 45 slightly increases diabetes risk, it is not a strong factor alone.
    - **BMI (31)**: A BMI above 30 is classified as obese, which is a known risk factor for diabetes.
    - **Glucose Level (88 mg/dL)**: This is well within the normal fasting glucose range (70-99 mg/dL), indicating no immediate concern for hyperglycemia.
    - **Hypertension (Yes)**: While high blood pressure is associated with metabolic disorders, it does not directly confirm diabetes without other contributing factors.

    Given that the glucose level is normal and there are no strong indicators of diabetes apart from BMI and hypertension, the prediction is: **Diabetes: No.**"""}
]

response = openClient.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

print(response.choices[0].message.content)


No. 

The reasoning behind this prediction is as follows:

- **Age (46)**: While age is a factor that can increase the risk of diabetes, at 46, this patient is not yet in a particularly high-risk age group compared to older populations.
  
- **BMI (31)**: This indicates obesity, which is a significant risk factor for developing type 2 diabetes. Obesity can lead to insulin resistance, which is a precursor to diabetes.

- **Glucose Level (88 mg/dL)**: A glucose level of 88 mg/dL is normal, falling within the standard fasting range of 70-99 mg/dL. This is encouraging, as it suggests that the patient's body is currently regulating blood sugar effectively.

- **Hypertension (Yes)**: High blood pressure is another risk factor associated with diabetes, but it alone does not confirm the disease; however, it does suggest a need for monitoring and lifestyle adjustments.

Combining these factors reveals that while the patient presents with a significant risk factor (obesity) and concurrent hypert

In [16]:
# Decision Tree Approach
messages = [
    {"role": "system", "content": "You are a doctor or an expert trained to use a decision tree for predicting Diabetes based on patient data. Follow the given decision tree strictly."},
    {"role": "user", "content": """Use the following decision tree to predict diabetes risk for the patient:

    Decision Tree:
      1. Age >= 45? → Yes → Proceed to step 2.
      2. BMI >= 30? → Yes → Proceed to step 3.
      3. Glucose >= 160? → Yes → Diabetes: Yes.
      4. Glucose < 160? → No → Diabetes: No.

    Patient Data: Age: 46, Gender: Male, BMI: 31, Glucose: 88, Hypertension: Yes

    Follow the decision tree step by step and provide the prediction."""},
    {"role": "assistant", "content": """Let's apply the decision tree:

    - Step 1: Age = 46 (≥ 45) → Yes, proceed to step 2.
    - Step 2: BMI = 31 (≥ 30) → Yes, proceed to step 3.
    - Step 3: Glucose = 88 (< 160) → No, proceed to step 4.
    - Step 4: Glucose < 160 → No.

    Final Prediction: Diabetes: No."""}
]

response = openClient.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

print(response.choices[0].message.content)


Based on the decision tree, here is the step-by-step application for the patient's data (Age: 46, BMI: 31, Glucose: 88):

1. **Step 1:** Age = 46 (≥ 45) → Yes, proceed to step 2.
2. **Step 2:** BMI = 31 (≥ 30) → Yes, proceed to step 3.
3. **Step 3:** Glucose = 88 (< 160) → No, proceed to step 4.
4. **Step 4:** Glucose < 160 → No.

**Final Prediction:** Diabetes: No.


**Process Synthetic Medical Data**

In [17]:
# Load datasets into a pandas DataFrame,
patients = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AIH/LLM/patients.csv")
conditions = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AIH/LLM/conditions.csv")
observations = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AIH/LLM/observations.csv")

In [18]:
patients.head(1)

,patient,birthdate,deathdate,ssn,drivers,passport,prefix,first,last,suffix,maiden,marital,race,ethnicity,gender,birthplace,address
0,4ee2c837-e60f-4c54-9fdf-8686bc70760b,1929-04-08,2029-11-11,999-78-5976,NaN,NaN,NaN,Rosamaria,Pfannerstill,NaN,NaN,NaN,black,dominican,F,Pittsfield MA US,18797 Karson Burgs Suite 444 Palmer Town MA 01...


In [19]:
conditions.head(1)

,start,stop,patient,encounter,code,description
0,2009-01-08,2009-01-21,71949668-1c2e-43ae-ab0a-64654608defb,4d451e22-a354-40c9-8b33-b6126158666d,10509002,Acute bronchitis (disorder)


In [20]:
observations.head(1)

,date,patient,encounter,code,description,value,units
0,2008-03-11,71949668-1c2e-43ae-ab0a-64654608defb,5114a5b4-64b8-47b2-82a6-0ce24aae0943,8302-2,Body Height,166.03,cm


In [21]:
"""
Data Preprocessing (Synthetic Medical Data)
"""

# Convert birthdate to age
patients['birthdate'] = pd.to_datetime(patients['birthdate'])
patients['age'] = (datetime.today() - patients['birthdate']).dt.days // 365  # Calculate age in years based on today's date

# Filter patients age > 100
patients = patients[patients['age'] < 100]

# Extract diabetes diagnosis from conditions.csv
conditions['Diabetes'] = conditions['description'] == "Diabetes"
diabetes_patients = conditions[conditions['Diabetes']]['patient'].unique()

# Label patients with diabetes
patients['Diabetes'] = patients['patient'].isin(diabetes_patients).astype(int)

# Convert 'value' to numeric, forcing errors to NaN (to handle any non-numeric values)
observations['value'] = pd.to_numeric(observations['value'], errors='coerce')

# Extract BMI, glucose, and hypertension from observations.csv
# For BMI, filter observations that contain "Body Mass Index" and compute the mean BMI per patient
bmi_df = observations[observations['description'].str.contains("Body Mass Index", na=False)]\
    .groupby('patient')['value'].mean().reset_index()
bmi_df.rename(columns={'value': 'BMI'}, inplace=True)  # Rename the 'value' column to 'BMI'

# For Glucose, filter observations that contain "Glucose" and compute the mean glucose level per patient
glucose_df = observations[observations['description'].str.contains("Glucose", na=False)]\
    .groupby('patient')['value'].mean().reset_index()
glucose_df.rename(columns={'value': 'Glucose_Level'}, inplace=True)  # Rename the 'value' column to 'Glucose_Level'

# For Hypertension, filter conditions that contain "Hypertension", then mark patients with Hypertension
hypertension_df = conditions[conditions['description'].str.contains("Hypertension", na=False)]\
    .groupby('patient').size().reset_index(name='Hypertension')
hypertension_df['Hypertension'] = 1  # If patient has hypertension, mark as 1

# Merge dataframes: Merge BMI, Glucose, and Hypertension data with the main patient dataframe
df = patients.merge(bmi_df, on='patient', how='left')  # Merge BMI data
df = df.merge(glucose_df, on='patient', how='left')  # Merge Glucose data
df = df.merge(hypertension_df, on='patient', how='left')  # Merge Hypertension data

# Rename columns for better readability
df.rename(columns={'age': 'Age'}, inplace=True)
df.rename(columns={'gender': 'Gender'}, inplace=True)

# Filter required columns
# Keep only necessary columns in the DataFrame
df = df[['Age', 'Gender', 'BMI', 'Glucose_Level', 'Hypertension', 'Diabetes']]

# Convert categorical variables
# Map 'M' to 1 and 'F' to 0 in the Gender column
df['Gender'] = df['Gender'].map({'M': 1, 'F': 0})

# Fill missing values with the mean of the respective columns
# This will replace NaNs in numeric columns with the column's mean value
df.fillna(df.mean(), inplace=True)

# Display the length of the resulting DataFrame
print(len(df))

# Show the first few rows of the resulting DataFrame
print(df.head())


1330
   Age  Gender        BMI  Glucose_Level  Hypertension  Diabetes
0   96       0  28.328828      88.362686           1.0         0
1    8       0  28.328828      88.362686           1.0         0
2   81       1  30.395000      84.400000           1.0         0
3   70       0  38.984444      76.777778           1.0         0
4   90       1  23.060000      87.000000           1.0         0


In [22]:
# Diabetes patient counts
print(df['Diabetes'].value_counts())

Diabetes
0    1247
1      83
Name: count, dtype: int64


In [23]:
# Data is inbalance so lets downsample data

# Separate the majority and minority classes based on the 'Diabetes' column
df_majority = df[df['Diabetes'] == 0]
df_minority = df[df['Diabetes'] == 1]

# Downsample the majority class to match the minority class size
df_majority_downsampled = resample(df_majority,
                                   replace=False,
                                   n_samples=len(df_minority),  # Match minority class size
                                   random_state=42)

# Combine the downsampled majority class with the minority class
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Display the new class distribution
print(f'New class distribution:\n{df_balanced["Diabetes"].value_counts()}')

New class distribution:
Diabetes
0    83
1    83
Name: count, dtype: int64


In [24]:
# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the shuffled balanced dataset
print(df_balanced.head(10))

   Age  Gender        BMI  Glucose_Level  Hypertension  Diabetes
0   99       0  28.328828      88.362686           1.0         1
1   59       0  30.260000     125.000000           1.0         1
2   36       1  18.490000      88.362686           1.0         0
3   34       0  23.828333      88.362686           1.0         0
4   98       1  28.328828      88.362686           1.0         1
5   80       0  27.531000      88.362686           1.0         0
6   90       0  28.328828      88.362686           1.0         1
7   64       1  35.568000      88.362686           1.0         0
8   87       1  30.165556      92.666667           1.0         1
9   93       1  28.328828      88.362686           1.0         1


In [25]:
df_balanced[df_balanced['Age'] < 50].head(10)

,Age,Gender,BMI,Glucose_Level,Hypertension,Diabetes
2,36,1,18.490000,88.362686,1.0,0
3,34,0,23.828333,88.362686,1.0,0
11,12,1,12.690000,88.362686,1.0,0
12,13,0,14.985000,88.362686,1.0,0
14,13,1,14.846000,88.362686,1.0,0
15,38,0,27.813333,88.362686,1.0,0
17,30,1,23.250000,88.362686,1.0,0
20,26,0,26.874444,88.362686,1.0,0
22,46,0,25.216667,88.362686,1.0,0
27,28,0,23.353000,88.362686,1.0,0


**Train Test Split**

In [26]:
# Train Test split

X = df_balanced.drop(columns=["Diabetes"])
y = df_balanced["Diabetes"]

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shape of the resulting datasets
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (132, 5)
X_test shape: (34, 5)
y_train shape: (132,)
y_test shape: (34,)


In [27]:
from torch.utils.data import Dataset

class DiabetesDataset(Dataset):
    def __init__(self, df):
        """
        Initializes the dataset with a DataFrame containing patient information.
        """
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        """
        Returns a formatted string for diabetes prediction.
        """
        column_names = [
            ("Age", "The patient's age is "),
            ("Gender", ". Gender is "),
            ("BMI", ". BMI is "),
            ("Glucose_Level", ". Glucose level is "),
            ("Hypertension", ". Hypertension status is ")
        ]

        # Convert row data into text description
        x_strs = [f"{col_desc}{self.df.iloc[index][col]}" for col, col_desc in column_names]
        x_str = ''.join(x_strs)

        # Replace numerical values with meaningful labels
        x_str = x_str.replace("Gender is 1", "Gender is Male")
        x_str = x_str.replace("Gender is 0", "Gender is Female")
        x_str = x_str.replace("Hypertension status is 1", "Hypertension status is Yes")
        x_str = x_str.replace("Hypertension status is 0", "Hypertension status is No")

        # Instruction for LLM
        x_str = ("Decide in a single word if the patient has diabetes based on the given details. "
                 "If the diabetes label is 1, the answer is 'Yes'. If 0, the answer is 'No'. " + x_str)
        x_str += ". Please decide whether the patient has diabetes: Yes or No."

        return x_str


In [28]:
test_ds = DiabetesDataset(X_test)

In [29]:
from tqdm import tqdm
import time

results = []
for prompt in tqdm(test_ds):
  response = openClient.chat.completions.create(
    model="gpt-4o-mini",  # Make sure this model is available
    messages=[
          {"role": "user", "content": prompt},
      ]
  )
  results.append(response.choices[0].message.content)
  time.sleep(3)

100%|██████████| 34/34 [02:19<00:00,  4.10s/it]


In [30]:
results

['No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes']

**Evaluating Diabetes Risk Prediction with OpenAPI: AUROC and AUPRC Metrics**

In [31]:
# Evaluating Diabetes Risk Prediction with OpenAPI: AUROC and AUPRC Metrics

# Convert predictions to binary (1 for "Yes", 0 for "No")
results_binary = list(map(lambda x: 1 if x == "Yes" else 0, results))

test_labels = y_test
test_pred = results_binary

auroc = roc_auc_score(test_labels, test_pred)
auprc = average_precision_score(test_labels, test_pred)
print('\nAUROC:', auroc, '\nAUPRC', auprc)


AUROC: 0.6214285714285713 
AUPRC 0.6588235294117647


**Using OpenAPI Embedding and Logistic Regression to Classify Diabetes Risk: Model Evaluation with AUROC and AUPRC**

In [32]:
def generate_embeddings(texts, model="text-embedding-ada-002"):
    embeddings = []

    for text in tqdm(texts):
        text = text.replace("\n", " ")  # Remove newline characters
        response = openClient.embeddings.create(
            input=[text], model=model
        )

        embeddings.append(response.data[0].embedding)

    return np.array(embeddings)

In [33]:
# Using OpenAPI Embedding and Logistic Regression to Classify Diabetes Risk: Model Evaluation with AUROC and AUPRC

# Train and test the performance
train_ds = DiabetesDataset(X_train)  # Example of a dataset, adjust as needed
embeddings = generate_embeddings(train_ds)  # Generate embeddings using OpenAPI

# Train the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(embeddings, y_train)  # Fit the model to the training data

# Generate embeddings for the test dataset
test_embeddings = generate_embeddings(test_ds)  # Embeddings for the test dataset
test_labels = y_test  # Actual labels for the test set

# Predict probabilities for the positive class (diabetes risk = 1)
test_pred = model.predict_proba(test_embeddings)[:, 1]

# Evaluate the model using AUROC and AUPRC
auroc = roc_auc_score(test_labels, test_pred)
auprc = average_precision_score(test_labels, test_pred)

# Output the results
print('\nAUROC:', auroc, '\nAUPRC', auprc)

100%|██████████| 34/34 [00:16<00:00,  2.06it/s]


AUROC: 0.8214285714285714 
AUPRC 0.8011603917537421
